In [2]:
import pandas as pd
import os
from multiprocessing import Process
import copy
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# Node의 부모 Class
class Node:
    def __init__(self, node_name):
        self.node_name = node_name

# Edge의 부모 Class
class Edge:
    def __init__(self, origin, dest):
        self.origin = origin
        self.dest = dest

# Airport Class
class Airport(Node):
    def __init__(self, airport_name):
        super().__init__(airport_name)

# Flight Class
class Flight(Edge):
    def __init__(self, index, origin, dest, origin_date, dest_date, flight_duration, flight_type):
        super().__init__(origin, dest)
        self.index = index
        self.origin_date = origin_date
        self.dest_date = dest_date
        self.flight_duration = flight_duration
        self.flight_type = flight_type

    def print(self):
        print(self.index, self.origin, self.origin_date, self.dest,
              self.dest_date, self.flight_duration, self.flight_type)
        

# Stack Node의 구조
class Stackitem:
    def __init__(self, flight: Flight, flights_sequence: list = None, landing_number = None):
        if flights_sequence:
            self.flight = flight
            flights_sequence.append(flight.index)
            self.flights_sequence = flights_sequence
            self.landing_number = landing_number + 1
            # self.time = time + dt.timedelta(flight.flight_duration)
        else:
            self.flight = flight
            self.flights_sequence = [flight.index]
            self.landing_number = 1
            # self.time = flight.flight_duration;

In [3]:
# Initial Setting - Test
flight_graph = dict()
flight_excel = pd.read_excel("flight.xlsx")
flight_excel.columns = ['INDEX', 'ORIGIN', 'ORIGIN_DATE', 'DEST','DEST_DATE', 'FLIGHT_DURATION','AIRCRAFT_TYPE' ]
airports_set = set()

In [4]:
# Graph insert Node
for i in range(100) :
    airports_set.add(flight_excel.loc[i]['ORIGIN'])
    airports_set.add(flight_excel.loc[i]['DEST'])
    
for airports in airports_set :
    flight_graph[airports] = list()

# Graph insert Edge
for i in range(100) :
    flight_graph[flight_excel.loc[i]['ORIGIN']].append(Flight(flight_excel.loc[i]['INDEX'], flight_excel.loc[i]['ORIGIN'], flight_excel.loc[i]['DEST'],
                                                              flight_excel.loc[i]['ORIGIN_DATE'], flight_excel.loc[i]['DEST_DATE'],
                                                              flight_excel.loc[i]['FLIGHT_DURATION'], flight_excel.loc[i]['AIRCRAFT_TYPE']))
    

In [5]:
print(len(airports_set))

5


In [6]:
pairing_set = list()

# clustering을 통한 flights set의 크기를 줄여야지 경우의 수가 줄어듬
# 하지만 그렇다면 시간을 어떻게 배분해야하는가? 기준은? 시간 → 기종? 조금더 생각해봐야 할 거 같다.

# 여기서 Node 시작에 따라 끝나는 경우는
# 만나는 Base가 같은경우가 있고,
# 만약 끝이 Base가 아닌데 갈 곳이 없는 경우가 있다.
# 그렇다면 모든 pairing의 모든 경우를 넣어도 모든 경우가 나온다!
# 

for airport in airports_set :
    # pairing_set.append(["-" * 10, airport, "-" * 10])

    for flight in flight_graph[airport] :
        # 항공편, 항공편의 순서를 넣는다
        stack_temp = [Stackitem(flight)]
        
        while stack_temp:
            # 현재 진행 중인 항공편의 정보
            flights_information_now = stack_temp.pop()

            pairing_set.append(copy.deepcopy(flights_information_now.flights_sequence))
            # 만약 해당 항공편이 진행됨으로서 base로 간다면 해당 항공편은 더 진행할 필요가 없음
            if (flights_information_now.flight.dest == flight.origin) :
                continue
            
            else :
                pairing_set[-1].append("DeadHead")

            # Search Space Extend
            for flight_next in flight_graph[flights_information_now.flight.dest] :
                # 항공기 기종이 같은지? 시간이 가능한지? (날짜가 같다면 시간까지 비교, 다르다면 날짜 비교)
                flight_type_equal = (flights_information_now.flight.flight_type == flight_next.flight_type)
                flight_time_possible = flights_information_now.flight.dest_date < flight_next.origin_date and (flight_next.origin_date - flights_information_now.flight.dest_date).seconds/3600 >= 3
                flight_day_possible = (flight_next.dest_date  - flight.origin_date).days <= 7
                flight_landing_possible = (flights_information_now.landing_number <= 3)

                # 기종이 다르거나 시간이 불가능하면 해당 항공편은 Search에 사용하지 않음
                if (not flight_type_equal or not flight_time_possible or not flight_day_possible or not flight_landing_possible) :
                    continue

                # 기종이 같고, 시간이 가능하다면 Search를 위해 stack에 추가(항공편의 순서까지)
                temp_list = copy.deepcopy(flights_information_now.flights_sequence)
                #if (flights_information_now.flight.dest_date < flight_next.origin_date) :
                    #for i in range(0, int(flight_next.origin_date.day - flights_information_now.flight.dest_date.day)) :
                        #temp_list.append("Layover")
                        
                stack_temp.append(Stackitem(flight_next, temp_list, flights_information_now.landing_number))
                

#for pairing in pairing_set :
    #print(pairing)

# 첫번째 방안 : 모든 항공편들의 순서를 저장한다
# 그렇다면 만들 수 있는(DeadHead, Layover, Feasible) Pairing의 모든 경우가 나온다.
# 하지만 이럴때는 시간이 n^n까지 나올 수 있는 것으로 보인다.
# 여기서 경우의 수를 줄이는 조건을 넣을 때
# 첫번째 - 항공기 종류
# 두번째 - 전 항공편의 도착시간보다 현 항공편의 출발시간이 느려야한다
# 세번째 - Crew의 Trun Time이 MMT보다 크거나 같아야 한다.(Slack Time은 어떻게 될려나?)
# feasible pairing은 좀 많다. -> 그럼 DeadHead가 존재하는 pairing은 넣어야하는 것인가?
# 지연전파? 존재한다?

# stack에 넣을 때 모든 정보를 넣어버린다면? 오히려 쉬워진다. 하지만 메모리를 많이 쓰므로 최적화가 필요함(조건을 생각해야 함)

In [7]:
import re

# 엑셀 파일명
filename = 'output.xlsx'

# 데이터프레임으로 변환
df = pd.DataFrame(pairing_set)

# 엑셀로 출력
df.to_excel(filename, index=False, header=False)

In [24]:
# 비행기 기종 : {크루원 수, 비행 수당, 기본급}
salary={}

salary_excel = pd.read_excel("salary.xlsx")
for index, row in salary_excel.iterrows():
    aircraft = row['AIRCRAFT']
    crew_num = row['CREW_NUM']
    flight_salary = row['FLIGHT_SALARY']
    base_salary = row['BASE_SALARY']
    layover_cost = row['BASE_SALARY']
    salary[aircraft] = {'CREW_NUM': crew_num, 'FLIGHT_SALARY': flight_salary, 'BASE_SALARY': base_salary,'LAYOVER_COST': layover_cost}


total = []

for pairing in pairing_set :
    crew_num = salary[flight_excel.loc[int(re.search(r'\d+', pairing[0]).group())-1]['AIRCRAFT_TYPE']]['CREW_NUM']
    base_salary = salary[flight_excel.loc[int(re.search(r'\d+', pairing[0]).group())-1]['AIRCRAFT_TYPE']]['BASE_SALARY']
    flight_salary = salary[flight_excel.loc[int(re.search(r'\d+', pairing[0]).group())-1]['AIRCRAFT_TYPE']]['FLIGHT_SALARY']
    layover_cost =  salary[flight_excel.loc[int(re.search(r'\d+', pairing[0]).group())-1]['AIRCRAFT_TYPE']]['LAYOVER_COST']
    layover_time = 0
    flight_time = flight_excel.loc[int(re.search(r'\d+', pairing[0]).group())]['FLIGHT_DURATION']
    deadhead=0
    for idx,pair in enumerate(pairing[1:],start=1):
        if(pair != "DeadHead") :
            n = int(re.search(r'\d+', pair).group())
            flight_time+=flight_excel.loc[n-1]['FLIGHT_DURATION']
            before_flight = flight_excel.loc[int(re.search(r'\d+', pairing[idx-1]).group())-1]['ORIGIN_DATE']
            current_flight = flight_excel.loc[int(re.search(r'\d+', pair).group())-1]['ORIGIN_DATE']
            layover_time += (current_flight-before_flight).seconds/3600
        else :
            #deadhead cost 정의하기
            deadhead_cost = 100
            deadhead = deadhead_cost * crew_num
        
            

    #print(pairing)
    #print('base_salary :',base_salary,'flight_salary :', flight_salary*flight_time,'layover_cost :',layover_cost*layover_time,'deadhead :',deadhead)
    total.append(base_salary+flight_salary*flight_time+layover_cost*layover_time)
    #print('Total: ',base_salary+flight_salary*flight_time+layover_cost*layover_time)
        
    #print(pairing)
    #print(total)



In [37]:
import gurobipy as gp
from gurobipy import GRB

def delta(pair, num):
    for p in pair:
      if p.startswith('F'):
            if p[1:] == str(num):
               return 1
    return 0



# 최적화 모델 생성
model = gp.Model('Crew_Pairing')

# 변수 추가
c = []
x = []

for j in range(100):
    c_j = model.addVar(vtype=GRB.CONTINUOUS, name='c_%s' % j)
    c.append(c_j)
    model.setAttr("Start", c_j, total[j])

for j in range(654):
    x_j = model.addVar(vtype=GRB.BINARY, name='x_%s' % j)
    x.append(x_j)
#   model.setAttr("Start", x_j, 1)
    


for j in range(100) :
    #비행편 마다의 제약 조건 추가?..
        model.addConstr(gp.quicksum(delta(pairing_set[i],j+1)*x[i] for i in range(654))==1, name='d_%s' % j)

# objective function 설정
obj = gp.quicksum(x[i] * c[i] for i in range(100))

model.setObjective(obj, GRB.MINIMIZE)
model.optimize()

if(model.Status==GRB.OPTIMAL) :
    print("최적해 찾음!!")
#print(model.ObjNVal)

# 변수 값 출력
for v in model.getVars():
    print(v.varName,v.getAttr)

#print(x,total)

# 제약 조건 슬랙 출력
for c in model.getConstrs():
   print(c.constrName,c.Slack)


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license - for non-commercial use only - registered to skxkswls@gmail.com
Optimize a model with 100 rows, 754 columns and 1676 nonzeros
Model fingerprint: 0x6ea6cee3
Model has 100 quadratic objective terms
Variable types: 100 continuous, 654 integer (654 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 15354.3 (0.02s)
Loaded user MIP start with objective 15354.3

Presolve removed 100 rows and 754 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 8 av